<a href="https://colab.research.google.com/github/ranadeepbhuyan/cancer-mri-analysis/blob/main/stateOfArt/IEGResUnet.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import cv2
import nibabel as nib
import os
import nibabel as nib

In [ ]:
import keras
import keras.backend as K
from keras.callbacks import CSVLogger
import tensorflow as tf
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report
from tensorflow.keras.models import *
from tensorflow.keras.layers import *
from tensorflow.keras.optimizers import *
from tensorflow.keras.callbacks import ModelCheckpoint, ReduceLROnPlateau, EarlyStopping, TensorBoard


In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
Data_path = r"/content/drive/MyDrive/PKG - UPENN-GBM NIfTI files/NIfTI-files/images_structural" +"/"

In [ ]:
Mask_path = r"/content/drive/MyDrive/PKG - UPENN-GBM NIfTI files/NIfTI-files/automated_segm" +"/"

In [ ]:
#fatching ids form data path
def pathListIntoIds(dirList):
    x = []
    for i in range(0,len(dirList)):
        x.append(dirList[i][dirList[i].rfind('/')+1:])
    return x


train_data_ids = [f.path for f in os.scandir(Data_path)]

training_data_ids = pathListIntoIds(train_data_ids);

In [ ]:
mask_data_ids = [f.path for f in os.scandir(Mask_path)]

mask_ids = pathListIntoIds(mask_data_ids);

In [ ]:
mask_ids.sort()
training_data_ids.sort()

In [ ]:
ids = []
for i in range(len(mask_ids)):
  test = mask_ids[i].split('_automated_approx_segm.nii')[0]
  ids.append(test)


In [ ]:
data_ids = []
for i in range(len(ids)):
  if ids[i] in training_data_ids:
    data_ids.append(ids[i])

In [ ]:
len(data_ids)

611

In [ ]:
only = []
for i in range(0,10):
  only.append(data_ids[i])

In [ ]:
train_test_ids, val_ids = train_test_split(only,test_size=0.2)
train_ids, test_ids = train_test_split(train_test_ids,test_size=0.15)

In [ ]:
VOLUME_SLICES = 24
IMG_SIZE = 128
required_shape = (IMG_SIZE,IMG_SIZE,IMG_SIZE)

In [ ]:
def add_padding(img_data, required_shape):
    # Ensure the dimensions of required_shape are (height, width, channels)
    required_height, required_width, required_length = required_shape

    # Calculate the amount of padding required on each side
    pad_x = required_height - img_data.shape[0]
    pad_y = required_width - img_data.shape[1]
    pad_z = required_length - img_data.shape[2]
    # Check if padding is needed
    if pad_x > 0 or pad_y > 0 or pad_z>0 :
        # Calculate the padding values for the top, bottom, left, and right sides
        pad_top = max(pad_x // 2, 0)
        pad_bottom = pad_x - pad_top
        pad_left = max(pad_y // 2, 0)
        pad_right = pad_y - pad_left
        pad_up = max(pad_z // 2, 0)
        pad_down = pad_z - pad_up

        # Create an empty array with the required shape
        padded_data = np.zeros((required_height, required_width, required_length), dtype=img_data.dtype)

        # Copy the original image data into the center of the padded array
        padded_data[pad_top:pad_top+img_data.shape[0], pad_left:pad_left+img_data.shape[1],pad_up:pad_up+img_data.shape[2]] = img_data
    else:
        # No padding needed; return the original image
        padded_data = img_data

    return padded_data


In [ ]:
class DataGenerator(tf.keras.utils.Sequence):
    'Generates data for Keras'
    def __init__(self, list_IDs, dim=(IMG_SIZE,IMG_SIZE,IMG_SIZE), batch_size = 1, n_channels = 3, shuffle=True):
        'Initialization'
        self.dim = dim
        self.batch_size = batch_size
        self.list_IDs = list_IDs
        self.n_channels = n_channels
        self.shuffle = shuffle
        self.on_epoch_end()

    def __len__(self):
        'Denotes the number of batches per epoch'
        return int(np.floor(len(self.list_IDs) / self.batch_size))

    def __getitem__(self, index):
        'Generate one batch of data'
        # Generate indexes of the batch
        indexes = self.indexes[index*self.batch_size:(index+1)*self.batch_size]
        # Find list of IDs
        Batch_ids = [self.list_IDs[k] for k in indexes]

        # Generate data
        X, y = self.__data_generation(Batch_ids)

        return X, y

    def on_epoch_end(self):
        'Updates indexes after each epoch'
        self.indexes = np.arange(len(self.list_IDs))
        if self.shuffle == True:
            np.random.shuffle(self.indexes)

    def __data_generation(self, Batch_ids):
        'Generates data containing batch_size samples' # X : (n_samples, *dim, n_channels)
        # Initialization
        X = np.zeros((self.batch_size*VOLUME_SLICES, *self.dim, self.n_channels))
        Y = np.zeros((self.batch_size*VOLUME_SLICES, *self.dim))
              # Generate data
        for c, i in enumerate(Batch_ids):
            data_path1 = os.path.join(Data_path, f'{i}/{i}_FLAIR.nii');
            flair_data = nib.load(data_path1).get_fdata()
            flair = add_padding(flair_data, required_shape)

            data_path1 = os.path.join(Data_path, f'{i}/{i}_T1GD.nii');
            t1gd_data = nib.load(data_path1).get_fdata()
            t1gd = add_padding(t1gd_data, required_shape)

            data_path1 =  os.path.join(Data_path, f'{i}/{i}_T2.nii');
            t2_data = nib.load(data_path1).get_fdata()
            t2 = add_padding(t2_data, required_shape)

            data_path1 = os.path.join(Mask_path, f'{i}_automated_approx_segm.nii');
            seg_data = nib.load(data_path1).get_fdata()
            seg = add_padding(seg_data, required_shape)

            slice_w = 25

            for j in range(VOLUME_SLICES):
                X[j +VOLUME_SLICES*c,:,:,:,0] = cv2.resize(flair[:,:,flair.shape[0]//4 +j*4], (IMG_SIZE, IMG_SIZE));
                X[j +VOLUME_SLICES*c,:,:,:,1] = cv2.resize(t1gd[:,:,t1gd.shape[0]//4 +j*4], (IMG_SIZE, IMG_SIZE));
                X[j +VOLUME_SLICES*c,:,:,:,2] = cv2.resize(t2[:,:,t2.shape[0]//4 +j*4], (IMG_SIZE, IMG_SIZE));

                Y[j +VOLUME_SLICES*c,:,:,:] = cv2.resize(seg[:,:,seg.shape[0]//4 +j*4], (IMG_SIZE, IMG_SIZE));


        Y[Y==4] = 3;
        mask = tf.one_hot(Y, 4);

        return X/np.max(X), mask

In [ ]:
training_generator = DataGenerator(train_ids)
valid_generator = DataGenerator(val_ids)
test_generator = DataGenerator(test_ids)

In [ ]:
def IEG(input, nfilters):
  conv1 = Conv3D(filters=nfilters, kernel_size=(3, 3, 3),dilation_rate=(3, 3, 3), padding='same', kernel_initializer='he_normal')(input)
  conv1 = BatchNormalization()(conv1)
  conv2 = Conv3D(filters=nfilters, kernel_size=(3, 3, 3),dilation_rate=(3, 3, 3), padding='same', kernel_initializer='he_normal')(conv1)

  conv3 = Conv3D(filters=nfilters, kernel_size=(3, 3, 3),dilation_rate=(3, 3, 3), padding='same', kernel_initializer='he_normal')(input)
  conv3 = BatchNormalization()(conv3)

  add_conv = Add()([conv2, conv3])

  return add_conv

In [ ]:
def residual_block(input, nfilters, f):
  conv1 = Conv3D(filters=nfilters, kernel_size=(3, 3, 3),dilation_rate=(f, f, f), activation = LeakyReLU(alpha=0.01),padding='same', kernel_initializer='he_normal')(input)
  conv1 = BatchNormalization()(conv1)
  conv2 = Conv3D(filters=nfilters, kernel_size=(3, 3, 3),dilation_rate=(f, f, f), activation = LeakyReLU(alpha=0.01), padding='same', kernel_initializer='he_normal')(conv1)
  conv2 = BatchNormalization()(conv2)

  conv3 = Conv3D(filters=nfilters, kernel_size=(1, 1, 1),dilation_rate=(f, f, f),activation = LeakyReLU(alpha=0.01), padding='same', kernel_initializer='he_normal')(input)
  conv3 = BatchNormalization()(conv3)

  y = Add()([conv2, conv3])
  return y

In [ ]:
def IEGResUnet(img_height, img_width, img_length, nclasses=None):

    def encoder_block(input_tensor, nfilters, dilation_rate):

      x = residual_block(input_tensor, nfilters, dilation_rate)
      x = BatchNormalization()(x)
      x = MaxPooling3D(pool_size=(2, 2, 2))(x)
      return x

    def decoder_block(input_tensor, nfilters):

      x = Conv3DTranspose(filters=nfilters, kernel_size=(3, 3, 3), strides=(2, 2, 2), padding='same', kernel_initializer='he_normal')(input_tensor)
      x = Activation('relu')(x)
      return x

    # f is number of filters
    filters = 4
    #input
    input_layer = Input(shape=(img_height, img_width, img_length, 3))

    # 1st encoder
    encoder_1 = encoder_block(input_layer, filters, 1)
    print(encoder_1.shape)
    # 2nd encoder
    encoder_2 = encoder_block(encoder_1, filters*2, 2)
    print(encoder_2.shape)

    # 3rd encoder
    encoder_3 = encoder_block(encoder_2, filters*4, 3)
    print(encoder_3.shape)

    # 4th encoder
    encoder_4 = encoder_block(encoder_3, filters*8, 4)
    print(encoder_4.shape)

    #bottom layer
    bottom_layer_1 = Conv3D(filters=filters*16, kernel_size=(3, 3, 3), activation = 'relu', padding='same', kernel_initializer='he_normal')(encoder_4)
    bottom_layer_1 = BatchNormalization()(bottom_layer_1)
    bottom_layer_1 = MaxPooling3D(pool_size=(2, 2, 2))(bottom_layer_1)
    bottom_layer_2 = Conv3D(filters=filters*16, kernel_size=(3, 3, 3), activation = 'relu', padding='same', kernel_initializer='he_normal')(bottom_layer_1)
    print(bottom_layer_2.shape)

    #4th decode
    upsampling_4 = decoder_block(bottom_layer_2, filters*8)
    decoder_add1_4 = Add()([encoder_4, upsampling_4])
    decoder_IEG_4 = IEG(decoder_add1_4, filters*8)
    decoder_add2_4 = Add()([upsampling_4, decoder_IEG_4])
    decoder_4 = residual_block(decoder_add2_4, filters*8, 4)
    print(bottom_layer_2.shape)

    #3rd decode
    upsampling_3 = decoder_block(decoder_4, filters*4)
    upsampling_IEG_3 = decoder_block(decoder_IEG_4, filters*4)
    decoder_add1_3 = Add()([encoder_3, upsampling_IEG_3])
    decoder_IEG_3 = IEG(decoder_add1_3, filters*4)
    decoder_add2_3 = Add()([upsampling_3, decoder_IEG_3])
    decoder_3 = residual_block(decoder_add2_3, filters*4, 3)
    print(decoder_3.shape)


    #2nd decode
    upsampling_2 = decoder_block(decoder_3, filters*2)
    upsampling_IEG_2 = decoder_block(decoder_IEG_3,filters*2)
    decoder_add1_2 = Add()([encoder_2, upsampling_IEG_2])
    decoder_IEG_2 = IEG(decoder_add1_2, filters*2)
    decoder_add2_2 = Add()([upsampling_2, decoder_IEG_2])
    decoder_2 = residual_block(decoder_add2_2, filters*2, 2)
    print(decoder_2.shape)


    #1st decode
    upsampling_1 = decoder_block(decoder_2,filters)
    upsampling_IEG_1 = decoder_block(decoder_IEG_2, filters)
    decoder_add1_1 = Add()([encoder_1, upsampling_IEG_1])
    decoder_IEG_1 = IEG(decoder_add1_1, filters)
    decoder_add2_1 = Add()([upsampling_1, decoder_IEG_1])
    decoder_1 = residual_block(decoder_add2_1, filters, 1)
    print(decoder_1.shape)


    #final layer
    final_upsampling = decoder_block(decoder_1,filters)
    final_output = Conv3DTranspose(filters=nclasses, kernel_size=(1, 1, 1), padding='same', kernel_initializer='he_normal')(final_upsampling)
    output_layer = Activation('softmax')(final_output)
    print(output_layer.shape)

    model = Model(inputs=input_layer, outputs=output_layer, name='IEGResUnet')
    return model

In [ ]:
# dice loss as defined above for 4 classes
def dice_coef(y_true, y_pred, smooth=1.0):
    class_num = 4
    for i in range(class_num):
        y_true_f = K.flatten(y_true[:,:,:,:,i])
        y_pred_f = K.flatten(y_pred[:,:,:,:,i])
        intersection = K.sum(y_true_f * y_pred_f)
        loss = ((2. * intersection + smooth) / (K.sum(y_true_f) + K.sum(y_pred_f) + smooth))
        if i == 0:
            total_loss = loss
        else:
            total_loss = total_loss + loss

    total_loss = total_loss / class_num
    return total_loss

In [ ]:
def mean_iou(y_true, y_pred, smooth=1.0):
    class_num = 4
    iou_total = 0

    for i in range(class_num):
        y_true_f = K.flatten(y_true[:,:,:,:,i])
        y_pred_f = K.flatten(y_pred[:,:,:,:,i])

        intersection = K.sum(K.abs(y_true_f * y_pred_f))
        union = K.sum(y_true_f) + K.sum(y_pred_f) - intersection

        iou = (intersection + smooth) / (union + smooth)
        iou_total += iou

    mean_iou = iou_total / class_num
    return mean_iou


In [ ]:
model = IEGResUnet(IMG_SIZE, IMG_SIZE, IMG_SIZE, nclasses=4)
model.compile(loss="categorical_crossentropy", optimizer=tf.keras.optimizers.Adam(learning_rate=0.001), metrics = ['accuracy',mean_iou, dice_coef] )

(None, 64, 64, 64, 4)
(None, 32, 32, 32, 8)
(None, 16, 16, 16, 16)
(None, 8, 8, 8, 32)
(None, 4, 4, 4, 64)
(None, 4, 4, 4, 64)
(None, 16, 16, 16, 16)
(None, 32, 32, 32, 8)
(None, 64, 64, 64, 4)
(None, 128, 128, 128, 4)


In [ ]:
model.summary()

Model: "IEGResUnet"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_3 (InputLayer)           [(None, 128, 128, 1  0           []                               
                                28, 3)]                                                           
                                                                                                  
 conv3d_76 (Conv3D)             (None, 128, 128, 12  328         ['input_3[0][0]']                
                                8, 4)                                                             
                                                                                                  
 batch_normalization_74 (BatchN  (None, 128, 128, 12  16         ['conv3d_76[0][0]']              
 ormalization)                  8, 4)                                                    

In [ ]:
earlystopping = EarlyStopping(monitor='val_loss',
                              mode='min',
                              verbose=1,
                              patience=15
                             )
checkpointer = ModelCheckpoint(filepath="IEGResUnet.hdf5",
                               verbose=1,
                               save_best_only=True
                              )
reduce_lr = ReduceLROnPlateau(monitor='val_loss',
                              mode='min',
                              verbose=1,
                              patience=5,
                              min_delta=0.0001,
                              factor=0.2
                             )

In [ ]:
history =  model.fit(training_generator,
                    epochs=10,
                    steps_per_epoch=len(train_ids),
                    callbacks= [checkpointer, reduce_lr, earlystopping],
                    validation_data = valid_generator
                    )

Epoch 1/10
